## Úvod do regresie - Lekcia 1

#### Uvedenie do kontextu

✅ Existuje mnoho typov regresných metód a výber tej správnej závisí od odpovede, ktorú hľadáte. Ak chcete predpovedať pravdepodobnú výšku osoby v určitom veku, použijete `lineárnu regresiu`, pretože hľadáte **číselnú hodnotu**. Ak vás zaujíma, či by určitý typ kuchyne mal byť považovaný za vegánsky alebo nie, hľadáte **priradenie kategórie**, a preto by ste použili `logistickú regresiu`. O logistickej regresii sa dozviete viac neskôr. Zamyslite sa nad niektorými otázkami, ktoré môžete klásť na základe údajov, a nad tým, ktorá z týchto metód by bola vhodnejšia.

V tejto časti budete pracovať s [malým datasetom o cukrovke](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Predstavte si, že by ste chceli otestovať liečbu pre pacientov s cukrovkou. Modely strojového učenia by vám mohli pomôcť určiť, ktorí pacienti by na liečbu reagovali lepšie, na základe kombinácií premenných. Dokonca aj veľmi základný regresný model, keď je vizualizovaný, môže ukázať informácie o premenných, ktoré by vám mohli pomôcť zorganizovať vaše teoretické klinické štúdie.

Tak teda, poďme sa pustiť do tejto úlohy!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Ilustrácia od @allison_horst</figcaption>

<!--![Ilustrácia od \@allison_horst](../../../../../../2-Regression/1-Tools/images/encouRage.jpg)<br>Ilustrácia od @allison_horst-->


## 1. Načítanie našej sady nástrojov

Pre túto úlohu budeme potrebovať nasledujúce balíky:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [kolekcia balíkov v R](https://www.tidyverse.org/packages), ktorá je navrhnutá tak, aby robila dátovú vedu rýchlejšou, jednoduchšou a zábavnejšou!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) je [rámec balíkov](https://www.tidymodels.org/packages/) určený na modelovanie a strojové učenie.

Môžete ich nainštalovať pomocou:

`install.packages(c("tidyverse", "tidymodels"))`

Nasledujúci skript skontroluje, či máte nainštalované balíky potrebné na dokončenie tohto modulu, a v prípade, že niektoré chýbajú, ich pre vás nainštaluje.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Teraz načítajme tieto úžasné balíky a sprístupnime ich v našej aktuálnej R relácii. (Toto je len na ilustráciu, `pacman::p_load()` to už za vás urobil)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Dataset o cukrovke

V tomto cvičení si precvičíme naše schopnosti regresie tým, že budeme robiť predpovede na datasete o cukrovke. [Dataset o cukrovke](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) obsahuje `442 vzoriek` údajov týkajúcich sa cukrovky, s 10 prediktívnymi premennými: `vek`, `pohlavie`, `index telesnej hmotnosti`, `priemerný krvný tlak` a `šesť meraní krvného séra`, ako aj výstupnú premennú `y`: kvantitatívne meranie progresie ochorenia jeden rok po základnom vyšetrení.

|Počet pozorovaní|442|
|----------------|:---|
|Počet prediktorov|Prvých 10 stĺpcov sú číselné prediktívne premenné|
|Výstup/Cieľ|Stĺpec 11 je kvantitatívne meranie progresie ochorenia jeden rok po základnom vyšetrení|
|Informácie o prediktoroch|- vek v rokoch
||- pohlavie
||- bmi index telesnej hmotnosti
||- bp priemerný krvný tlak
||- s1 tc, celkový cholesterol v sére
||- s2 ldl, lipoproteíny s nízkou hustotou
||- s3 hdl, lipoproteíny s vysokou hustotou
||- s4 tch, celkový cholesterol / HDL
||- s5 ltg, pravdepodobne logaritmus hladiny triglyceridov v sére
||- s6 glu, hladina cukru v krvi|

> 🎓 Pamätajte, že ide o učenie s učiteľom (supervised learning), a potrebujeme cieľovú premennú nazvanú 'y'.

Predtým, než budete môcť manipulovať s údajmi v R, musíte údaje importovať do pamäte R alebo vytvoriť spojenie s údajmi, ktoré R môže použiť na vzdialený prístup k údajom.

> Balík [readr](https://readr.tidyverse.org/), ktorý je súčasťou Tidyverse, poskytuje rýchly a priateľský spôsob, ako načítať obdĺžnikové údaje do R.

Teraz načítajme dataset o cukrovke z poskytnutého zdrojového URL: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Tiež vykonáme kontrolu údajov pomocou `glimpse()` a zobrazíme prvých 5 riadkov pomocou `slice()`.

Predtým, než budeme pokračovať ďalej, predstavme si niečo, s čím sa často stretnete v kóde R 🥁🥁: operátor pipe `%>%`

Operátor pipe (`%>%`) vykonáva operácie v logickej postupnosti tým, že posúva objekt ďalej do funkcie alebo výrazu. Môžete si operátor pipe predstaviť ako "a potom" vo vašom kóde.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` nám ukazuje, že tieto dáta obsahujú 442 riadkov a 11 stĺpcov, pričom všetky stĺpce majú dátový typ `double`.

<br>

> glimpse() a slice() sú funkcie v [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, ktorý je súčasťou Tidyverse, predstavuje gramatiku manipulácie s dátami a poskytuje konzistentnú sadu slovies, ktoré vám pomôžu riešiť najbežnejšie výzvy pri práci s dátami.

<br>

Teraz, keď máme dáta, zamerajme sa na jednu vlastnosť (`bmi`), ktorú použijeme ako cieľ pre toto cvičenie. To si bude vyžadovať výber požadovaných stĺpcov. Ako to môžeme urobiť?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) nám umožňuje *vybrať* (a prípadne premenovať) stĺpce v dátovom rámci.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Tréningové a testovacie dáta

V supervidovanom učení je bežnou praxou *rozdeliť* dáta na dva podmnožiny; (zvyčajne väčšiu) množinu, s ktorou sa model trénuje, a menšiu „rezervnú“ množinu, na ktorej sa overí, ako model fungoval.

Teraz, keď máme dáta pripravené, môžeme zistiť, či nám stroj dokáže pomôcť určiť logické rozdelenie čísel v tejto dátovej sade. Môžeme použiť balík [rsample](https://tidymodels.github.io/rsample/), ktorý je súčasťou rámca Tidymodels, na vytvorenie objektu obsahujúceho informácie o *tom, ako* rozdeliť dáta, a potom ďalšie dve funkcie rsample na extrakciu vytvorených tréningových a testovacích množín:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Natrénujte lineárny regresný model pomocou Tidymodels

Teraz sme pripravení natrénovať náš model!

V Tidymodels špecifikujete modely pomocou `parsnip()` tým, že definujete tri koncepty:

-   **Typ** modelu rozlišuje medzi modelmi, ako sú lineárna regresia, logistická regresia, modely rozhodovacích stromov a podobne.

-   **Režim** modelu zahŕňa bežné možnosti, ako sú regresia a klasifikácia; niektoré typy modelov podporujú obe možnosti, zatiaľ čo iné majú iba jeden režim.

-   **Engine** modelu je výpočtový nástroj, ktorý sa použije na natrénovanie modelu. Často ide o balíky v R, ako napríklad **`"lm"`** alebo **`"ranger"`**

Tieto informácie o modelovaní sú zachytené v špecifikácii modelu, takže si jednu vytvorme!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Po tom, čo je model *špecifikovaný*, môže byť `odhadnutý` alebo `natrénovaný` pomocou funkcie [`fit()`](https://parsnip.tidymodels.org/reference/fit.html), zvyčajne s použitím vzorca a nejakých údajov.

`y ~ .` znamená, že budeme prispôsobovať `y` ako predpovedanú hodnotu/cieľ, vysvetlenú všetkými prediktormi/vlastnosťami, teda `.` (v tomto prípade máme iba jeden prediktor: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Z modelového výstupu môžeme vidieť koeficienty naučené počas tréningu. Predstavujú koeficienty priamky najlepšieho prispôsobenia, ktorá nám poskytuje najnižšiu celkovú chybu medzi skutočnou a predpovedanou premennou.
<br>

## 5. Predikcia na testovacej množine

Teraz, keď sme natrénovali model, môžeme ho použiť na predikciu progresie ochorenia y pre testovaciu množinu dát pomocou [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Toto sa použije na vykreslenie čiary medzi skupinami dát.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Hurá! 💃🕺 Práve sme natrénovali model a použili ho na vytváranie predpovedí!

Pri vytváraní predpovedí je v súlade s konvenciou tidymodels vždy vytvoriť tibble/dátový rámec s výsledkami, ktoré majú štandardizované názvy stĺpcov. To umožňuje jednoducho skombinovať pôvodné dáta a predpovede do použiteľného formátu pre ďalšie operácie, ako je napríklad vizualizácia.

`dplyr::bind_cols()` efektívne spája viacero dátových rámcov podľa stĺpcov.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Zobrazenie výsledkov modelovania

Teraz je čas vidieť to vizuálne 📈. Vytvoríme bodový graf všetkých hodnôt `y` a `bmi` z testovacej množiny a potom použijeme predpovede na nakreslenie čiary na najvhodnejšom mieste medzi skupinami údajov modelu.

R má niekoľko systémov na tvorbu grafov, ale `ggplot2` je jedným z najelegantnejších a najvšestrannejších. Umožňuje vám vytvárať grafy **kombinovaním nezávislých komponentov**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Zamyslite sa trochu nad tým, čo sa tu deje. Priama čiara prechádza cez množstvo malých bodov údajov, ale čo presne robí? Vidíte, ako by ste mali byť schopní použiť túto čiaru na predpovedanie, kam by mal nový, nevidený údaj zapadnúť vo vzťahu k osi y grafu? Skúste slovami vyjadriť praktické využitie tohto modelu.

Gratulujeme, vytvorili ste svoj prvý model lineárnej regresie, urobili ste s ním predpoveď a zobrazili ste ju v grafe!



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby na automatický preklad [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, upozorňujeme, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre dôležité informácie odporúčame profesionálny ľudský preklad. Nezodpovedáme za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
